<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance


    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-24 10:46:51
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 11.39 K (0.08%)
Current PnL: -25.14 L (-16.24%)
CY Booked + Current PnL: -10.85 L (-7.01%)
-------------------
Total profit:  1.66 L
Total loss:  -26.81 L
-------------------
Total Booked + Current PnL: 16.13 L (12.62%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.07%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 89.78 L (63.26%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,1.18,9.67,9.14,19.69,19220.0,18543.0,210285.0,-1.08,58.0,X-MC,2.49,78.0,0.96,1.49,17.13,XY25,NTT,AC
77,TTKPRESTIG,770.00,-0.08,-19.19,23.86,0.09,19431.0,-19341.0,81436.0,89.11,38.0,M-SC,2.62,253.0,-1.00,0.58,6.99,OX40N,NTT,DURABLES
50,MASFIN,398.61,-0.23,-1.33,23.70,22.05,22912.0,-1305.0,96675.0,-15.09,56.0,H-SC,6.64,164.0,-0.06,0.68,40.52,XR,ATH,FINANCE
43,ITC,452.00,-0.00,-0.16,10.96,10.78,26296.0,-389.0,239929.0,-40.34,58.0,X-LC,2.01,5.0,-0.01,1.70,5.07,X40,NTT,FMCG
52,NESTLEIND,1377.00,0.25,11.05,9.32,21.40,27470.0,29320.0,294746.0,3.02,57.0,X-LC,5.53,12.0,1.07,2.09,18.42,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SFL,1287.00,2.74,-40.98,114.55,26.63,177279.0,-107457.0,154761.0,17.29,49.0,M-SC,12.31,239.0,-0.61,1.10,5.95,XY24,NTT,MISC
26,GILLETTE,11206.78,2.59,-2.79,36.22,32.42,89395.0,-7086.0,246810.0,-20.08,47.0,X-SC,10.72,85.0,-0.08,1.75,11.89,X40,ATH,FMCG
74,TITAGARH,1548.00,2.29,-19.39,80.79,45.74,157722.0,-46952.0,195225.0,7.41,62.0,H-SC,6.41,174.0,-0.30,1.38,26.97,XY24,NTT,ENGINEERING
0,5PAISA,593.00,2.20,-38.55,83.73,12.90,112419.0,-84232.0,134264.0,98.32,54.0,H-SC,12.66,161.0,-0.75,0.95,11.64,OX40N,NTT,FINANCE
22,DIXON,18931.72,2.08,-12.77,44.53,26.07,75829.0,-24934.0,170287.0,-57.60,38.0,X-MC,12.37,56.0,-0.33,1.21,6.06,X40N,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,-1.05,-38.54,127.14,39.61,80098.0,-39500.0,63000.0,-58.53,31.0,H-SC,31.09,135.0,-0.49,0.45,0.00,AR,ATH,MISC
81,VBL,671.64,-0.98,-3.64,40.79,35.67,124148.0,-11484.0,304358.0,-15.14,53.0,X-LC,3.46,4.0,-0.09,2.15,9.71,X40N,ATH,FMCG
75,TMPV,600.00,-0.95,-26.26,66.76,22.98,111213.0,-59311.0,166587.0,-23.08,50.0,X-LC,6.40,3.0,-0.53,1.18,4.78,XY24,NTT,AUTO
78,UNITDSPR,1644.00,-0.93,4.93,15.09,20.77,37074.0,11541.0,245685.0,-4.14,50.0,X-MC,1.97,62.0,0.31,1.74,12.19,X40N,NTT,BREWERIES
83,WHIRLPOOL,2270.00,-0.84,-25.12,142.94,81.92,122878.0,-28833.0,85965.0,-56.08,31.0,M-SC,21.75,236.0,-0.23,0.61,2.55,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,1.97,0.41,18.74,19.23,28311.0,610.0,151071.0,18.42,78.0,L-LC,2.54,179.0,0.02,1.07,21.77,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.77,-2.92,114.18,107.91,163650.0,-4318.0,143326.0,-20.96,51.0,M-SC,11.87,216.0,-0.03,1.01,6.87,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.07,-17.00,20.51,0.02,29715.0,-29677.0,144882.0,120.98,40.0,M-SC,11.39,234.0,-1.00,1.03,26.02,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.08,-19.19,23.86,0.09,19431.0,-19341.0,81436.0,89.11,38.0,M-SC,2.62,253.0,-1.00,0.58,6.99,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.63,-22.25,69.82,32.03,71563.0,-29339.0,102496.0,-15.87,60.0,H-SC,8.96,148.0,-0.41,0.73,14.80,OX40N,NTT,IT
66,SIS,528.00,0.62,-23.65,58.82,21.26,49865.0,-26257.0,84775.0,2000.52,53.0,H-SC,4.78,166.0,-0.53,0.60,14.54,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,1.97,0.41,18.74,19.23,28311.0,610.0,151071.0,18.42,78.0,L-LC,2.54,179.0,0.02,1.07,21.77,XY25,ATH,MINING
84,WIPRO,420.00,-0.77,10.46,55.96,72.27,104737.0,17716.0,187164.0,-5.17,69.0,M-LC,3.90,99.0,0.17,1.33,17.28,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.77,10.46,55.96,72.27,104737.0,17716.0,187164.0,-5.17,69.0,M-LC,3.90,99.0,0.17,1.33,17.28,XR,NTT,IT
25,FINCABLES,1641.55,-0.77,-2.92,114.18,107.91,163650.0,-4318.0,143326.0,-20.96,51.0,M-SC,11.87,216.0,-0.03,1.01,6.87,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.52,-3.40,114.00,106.72,135824.0,-4192.0,119144.0,-52.15,43.0,H-SC,4.48,139.0,-0.03,0.84,19.18,AR,ATH,MISC
6,ASIANTILES,137.00,-0.75,-3.46,88.19,81.67,79868.0,-3247.0,90563.0,8155.56,69.0,L-SC,12.85,271.0,-0.04,0.64,75.38,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.23,-1.33,23.70,22.05,22912.0,-1305.0,96675.0,-15.09,56.0,H-SC,6.64,164.0,-0.06,0.68,40.52,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.45,-18.26,90.21,55.47,179387.0,-44431.0,198855.0,-27.60,30.0,X-MC,15.61,64.0,-0.25,1.41,1.89,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.18,-18.23,75.27,43.32,100245.0,-29692.0,133180.0,-29.79,31.0,X-SC,5.98,90.0,-0.30,0.94,2.26,X40N,ATH,ELECTRICAL
8,AWL,485.00,-0.00,-24.25,102.63,53.50,234591.0,-73172.0,228579.0,-63.37,33.0,X-MC,5.89,58.0,-0.31,1.62,1.53,XY24,NTT,FMCG
3,ACC,3906.00,-0.58,-26.71,123.97,64.15,216204.0,-63551.0,174400.0,-57.19,34.0,X-SC,4.20,82.0,-0.29,1.23,0.00,XY24,BTT,CEMENT
20,DABUR,735.00,0.14,-3.45,48.83,43.70,116956.0,-8551.0,239517.0,-11.44,36.0,X-MC,2.12,72.0,-0.07,1.70,11.08,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.19,-2.89,24.75,21.14,54640.0,-6573.0,220766.0,-9.80,47.0,X-MC,1.03,74.0,-0.12,1.56,24.44,XY24,NTT,PAINTS
15,CAMS,950.0,1.76,1.36,22.80,24.48,58206.0,3432.0,255288.0,-80.65,54.0,X-SC,1.26,86.0,0.06,1.81,24.82,X40N,NTT,MISC
78,UNITDSPR,1644.0,-0.93,4.93,15.09,20.77,37074.0,11541.0,245685.0,-4.14,50.0,X-MC,1.97,62.0,0.31,1.74,12.19,X40N,NTT,BREWERIES
43,ITC,452.0,-0.00,-0.16,10.96,10.78,26296.0,-389.0,239929.0,-40.34,58.0,X-LC,2.01,5.0,-0.01,1.70,5.07,X40,NTT,FMCG
20,DABUR,735.0,0.14,-3.45,48.83,43.70,116956.0,-8551.0,239517.0,-11.44,36.0,X-MC,2.12,72.0,-0.07,1.70,11.08,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.58,-26.71,123.97,64.15,216204.0,-63551.0,174400.0,-57.19,34.0,X-SC,4.20,82.0,-0.29,1.23,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,-0.09,-20.10,77.11,41.51,162255.0,-52945.0,210420.0,-4.76,39.0,X-MC,7.86,61.0,-0.33,1.49,0.80,XY25,ATH,FMCG
11,BATAINDIA,2096.00,-0.06,-38.73,120.68,35.20,95136.0,-49837.0,78833.0,3.39,36.0,X-SC,17.01,92.0,-0.52,0.56,1.12,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.00,-24.25,102.63,53.50,234591.0,-73172.0,228579.0,-63.37,33.0,X-MC,5.89,58.0,-0.31,1.62,1.53,XY24,NTT,FMCG
9,BAJAJHFL,181.50,-0.45,-18.26,90.21,55.47,179387.0,-44431.0,198855.0,-27.60,30.0,X-MC,15.61,64.0,-0.25,1.41,1.89,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.69,-27.76,97.73,42.84,45894.0,-18046.0,46960.0,-52.62,52.0,X-SC,33.80,83.0,-0.39,0.33,6.24,XY24,NTT,MISC
59,RELAXO,1176.00,1.13,-47.16,188.24,52.31,144388.0,-68456.0,76704.0,-43.76,48.0,X-SC,7.31,91.0,-0.47,0.54,2.67,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.06,-38.73,120.68,35.20,95136.0,-49837.0,78833.0,3.39,36.0,X-SC,17.01,92.0,-0.52,0.56,1.12,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.65,-1.45,25.90,24.08,32404.0,-1838.0,125112.0,-9.50,46.0,X-SC,6.44,89.0,-0.06,0.89,17.88,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.18,-18.23,75.27,43.32,100245.0,-29692.0,133180.0,-29.79,31.0,X-SC,5.98,90.0,-0.30,0.94,2.26,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.05,-8.94,32.56,20.71,112145.0,-33810.0,344427.0,-21.68,70.0,X-LC,3.21,1.0,-0.30,2.44,15.09,X40,ATH,IT
41,INFY,1972.00,-0.40,13.11,18.68,34.24,66112.0,41033.0,353921.0,-12.06,67.0,X-LC,6.15,2.0,0.62,2.51,22.48,X40,NTT,IT
75,TMPV,600.00,-0.95,-26.26,66.76,22.98,111213.0,-59311.0,166587.0,-23.08,50.0,X-LC,6.40,3.0,-0.53,1.18,4.78,XY24,NTT,AUTO
81,VBL,671.64,-0.98,-3.64,40.79,35.67,124148.0,-11484.0,304358.0,-15.14,53.0,X-LC,3.46,4.0,-0.09,2.15,9.71,X40N,ATH,FMCG
43,ITC,452.00,-0.00,-0.16,10.96,10.78,26296.0,-389.0,239929.0,-40.34,58.0,X-LC,2.01,5.0,-0.01,1.70,5.07,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.38,-37.02,111.00,32.89,53976.0,-28580.0,48627.0,-700.55,56.0,L-MC,6.83,259.0,-0.53,0.34,33.91,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-0.23,-57.02,132.81,0.07,78307.0,-78215.0,58962.0,2061.62,70.0,L-SC,4.04,268.0,-1.00,0.42,46.64,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-0.75,-3.46,88.19,81.67,79868.0,-3247.0,90563.0,8155.56,69.0,L-SC,12.85,271.0,-0.04,0.64,75.38,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.23,-1.33,23.70,22.05,22912.0,-1305.0,96675.0,-15.09,56.0,H-SC,6.64,164.0,-0.06,0.68,40.52,XR,ATH,FINANCE
13,BSOFT,831.70,-0.41,-17.69,83.19,50.79,96309.0,-24883.0,115770.0,5.53,72.0,H-SC,8.47,151.0,-0.26,0.82,34.79,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.41,-17.69,83.19,50.79,96309.0,-24883.0,115770.0,5.53,72.0,H-SC,8.47,151.0,-0.26,0.82,34.79,XR,ATH,IT
56,RAJESHEXPO,518.00,-0.23,-57.02,132.81,0.07,78307.0,-78215.0,58962.0,2061.62,70.0,L-SC,4.04,268.0,-1.00,0.42,46.64,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-0.75,-3.46,88.19,81.67,79868.0,-3247.0,90563.0,8155.56,69.0,L-SC,12.85,271.0,-0.04,0.64,75.38,XR,NTT,CERAMICS
61,ROUTE,2227.21,-0.61,-46.06,213.43,69.05,153181.0,-61295.0,71771.0,-56.49,54.0,H-SC,20.62,140.0,-0.40,0.51,10.69,SR,ATH,IT
17,COALINDIA,484.83,1.97,0.41,18.74,19.23,28311.0,610.0,151071.0,18.42,78.0,L-LC,2.54,179.0,0.02,1.07,21.77,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.82
2,50,76.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.03
MC    30.72
LC    24.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.79
X40      23.03
X40N     12.94
XR        9.93
XY25      9.15
AR        8.90
OX40N     7.53
SR        1.00
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.20
H-SC    23.85
X-LC    20.76
M-SC    11.58
X-SC     8.05
H-MC     4.79
L-SC     1.55
M-MC     1.39
M-LC     1.33
H-LC     1.11
L-LC     1.07
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.21,-7.34,41.85
IT,13.15,-13.31,70.29
FINANCE,9.57,-17.93,69.23
MISC,7.31,-25.83,77.30
ELECTRICAL,5.96,-11.55,52.41
PAINTS,5.79,-13.00,29.79
INSURANCE,4.74,-1.20,35.84
PHARMA,4.04,-2.78,35.13
AUTO,2.87,-29.64,74.58


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3138513.0,21
AR,1315064.0,10
XR,1259562.0,13
X40,970997.0,14
X40N,856762.0,9
OX40N,730474.0,10
XY25,349245.0,6
SR,279397.0,2
MH,77610.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3610433.0,25
M-SC,1424216.0,15
X-MC,1408809.0,16
X-LC,793871.0,11
X-SC,781872.0,8
H-MC,401544.0,3
L-SC,243231.0,3
M-LC,104737.0,1
H-LC,77390.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1257906.0      6
           AR         903339.0      5
M-SC       XY24       803664.0      6
H-SC       XR         761279.0      7
X-MC       X40        485482.0      7
           XY24       406187.0      3
           X40N       335665.0      4
H-SC       OX40N      330902.0      4
M-SC       OX40N      321265.0      5
X-SC       X40N       302839.0      3
X-LC       X40        300984.0      5
X-SC       XY24       294502.0      3
H-SC       SR         279397.0      2
X-LC       X40N       218258.0      2
H-MC       AR         209694.0      2
           XY24       191850.0      1
X-SC       X40        184531.0      2
X-LC       XY24       184404.0      2
X-MC       XY25       181475.0      2
M-SC       XR         174646.0      2
L-SC       XR         164924.0      2
M-SC       AR         124641.0      2
M-LC       XR         104737.0      1
X-LC       XY25        90225.0      2
L-SC       OX40N       78307.0      1
H-SC       MH          77610.0      1
H-LC       AR          77390.0      1
L-MC       XR          53976.0      1
M-MC       XY25        49234.0      1
L-LC       XY25        28311.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
